In [1]:
import json
import pandas as pd
import numpy as np

In [2]:
df = pd.read_excel(
    r"word_list\japanese\NLT1.40_freq_list.xlsx",
    sheet_name="NLT 1.40頻度リスト",
)

# extract and rename columns
df = df[["レマ", "読み", "頻度"]].rename(
    columns={"レマ": "annotation", "読み": "word", "頻度": "count"}
)

# cleanse words
df = df[np.isfinite(df["count"])]
df = df.dropna()

# fix format
df["word"] = df["word"].astype(str)
df["count"] = df["count"].astype(int)
agg_functions = {"word": "first", "annotation": "first", "count": "sum"}
df = df.groupby(df["word"], as_index=False).aggregate(agg_functions)

# allowed alphabets
df = df[df["word"].str.isalpha()]

# frequent words
df = df[df["count"] > df["count"].quantile(0.5)]

df["word_len"] = df["word"].apply(len)
df = df.sort_values(by=["word"]).reset_index(drop=True)
df.head()

,word,annotation,count,word_len
0,ℓ,ℓ,564,1
1,々,々,73050,1
2,〆,〆,1170,1
3,っ,っ,133403,1
4,ゝ,ゝ,8109,1


In [3]:
df.dtypes

word          object
annotation    object
count          int32
word_len       int64
dtype: object

In [4]:
df.describe()

,count,word_len
count,3.714400e+04,37144.000000
mean,2.556308e+04,4.421979
std,5.607082e+05,1.347253
min,1.580000e+02,1.000000
25%,4.800000e+02,4.000000
50%,1.291000e+03,4.000000
75%,4.833250e+03,5.000000
max,5.726447e+07,13.000000


In [5]:
df[(df["word_len"] >= 3) & (df["word_len"] <= 5) & (df["count"] > 10)].describe()

,count,word_len
count,2.808800e+04,28088.000000
mean,1.171286e+04,4.005020
std,6.246941e+04,0.728651
min,1.580000e+02,3.000000
25%,5.110000e+02,3.000000
50%,1.398000e+03,4.000000
75%,5.135250e+03,5.000000
max,4.111651e+06,5.000000


In [6]:
df.query("'タビタビ' in word")

,word,annotation,count,word_len
19539,タビタビ,たびたび,11536,4


In [23]:
difficulties = {"easy": 0.96, "medium": 0.9, "hard": 0.7, "all": 0}
min_len = 4
max_len = 5

for name, base_percentile in difficulties.items():
    dfs_filtered: list[pd.DataFrame] = []
    lengths: list[str] = []
    for l in range(min_len, max_len + 1):
        df_len = df[df["word_len"] == l]
        # japanese longer words don't need larger vocab pool
        percentile = max(base_percentile - (l - min_len) * 0.02, 0)
        cutoff = df_len["count"].quantile(percentile)
        dfs_filtered.append(
            df_len[df_len["count"] > cutoff]
            .sort_values(by=["count"], ascending=False)
            .reset_index(drop=True)
        )
        lengths.append(f"len{l}: #{len(dfs_filtered[-1])}")
    df_filtered = pd.concat(dfs_filtered)

    print(", ".join(lengths))

    with open(rf"vocabs\ja\ja-{name}.json", "w") as f:
        data = df_filtered["word"].sort_values().tolist()
        json.dump(data, f)
        print(f"{f.name}: {len(df_filtered)} words")

    if name == "all":
        with open(rf"vocabs\ja\ja-annotations.json", "w") as f:
            data = {}
            for index, row in df_filtered.iterrows():
                data[row["word"]] = row["annotation"]
            json.dump(data, f)
            print(f"{f.name}: {len(df_filtered)} annotations")

len4: #527, len5: #452
vocabs\ja\ja-easy.json: 979 words
len4: #1318, len5: #904
vocabs\ja\ja-medium.json: 2222 words
len4: #3953, len5: #2409
vocabs\ja\ja-hard.json: 6362 words
len4: #13155, len5: #7517
vocabs\ja\ja-all.json: 20672 words
vocabs\ja\ja-annotations.json: 20672 annotations
